In [46]:
import pandas as pd
from pathlib import Path
import numpy as np
import alpaca_trade_api as tradeapi
from alpaca.data.historical import CryptoHistoricalDataClient
from alpaca.data.requests import CryptoBarsRequest
from alpaca.data.timeframe import TimeFrame

from finta import TA
from finta.utils import resample_calendar
import json
import hvplot.pandas
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from pandas.tseries.offsets import DateOffset
from sklearn.metrics import classification_report

In [47]:
# Alpaca for data
import alpaca_trade_api as api
from alpaca_trade_api.rest import TimeFrame

# pandas for analysis
import pandas as pd

# Plotly for charting
import plotly.graph_objects as go
import plotly.express as px

# Set default charting for pandas to plotly
pd.options.plotting.backend = "plotly"

In [79]:
import datetime as dt
today = dt.date.today()
# symbols we will be looking at
btc = "BTCUSD"
spy = "SPY"

# start dates and end dates for backtest
start_date = "2020-01-01"
end_date = today #can't use today timeframe for spy data
#end_date = '2023-02-04'

# time frame for backtests
timeframe = TimeFrame.Day

In [80]:
# Our API keys for Alpaca
API_KEY = "PK5CD8L7ME7T2V9P1IB0"
API_SECRET = "N0bSGJwVRW4jLt0Uhj11DZEXyp7tnKP19FbcCz1P"

# Setup instance of alpaca api
alpaca = api.REST(API_KEY, API_SECRET)

# # # Request historical bar data for SPY and BTC using Alpaca Data API
# for equities, use .get_bars
#spy_data = alpaca.get_bars(spy, timeframe, start_date, end_date).df

# for crypto, use .get_crypto_bars, from multiple exchanges
btc_data = alpaca.get_crypto_bars(btc, timeframe, start_date, end_date).df

btc_data = btc_data[btc_data['exchange'] == 'CBSE']

# display crypto bar data
display(btc_data)
#display(spy_data)

,exchange,open,high,low,close,volume,trade_count,vwap
timestamp,,,,,,,,
2020-01-01 06:00:00+00:00,CBSE,7174.33,7186.18,7089.00,7108.03,1357.688568,10937,7132.489436
2020-01-02 06:00:00+00:00,CBSE,6945.02,7281.24,6854.67,7203.04,4385.672853,19558,7073.061633
2020-01-03 06:00:00+00:00,CBSE,7203.04,7401.20,7176.07,7334.45,9593.744578,45872,7299.646322
2020-01-04 06:00:00+00:00,CBSE,7333.95,7398.00,7263.32,7348.63,3788.097227,28692,7325.605432
2020-01-05 06:00:00+00:00,CBSE,7355.40,7580.00,7343.17,7528.70,3362.380611,15608,7491.042016
...,...,...,...,...,...,...,...,...
2023-02-01 06:00:00+00:00,CBSE,23117.38,24262.18,22760.00,23809.86,21257.753630,505881,23372.248346
2023-02-02 06:00:00+00:00,CBSE,23809.87,24148.37,23367.87,23527.78,13127.923248,400407,23757.316321
2023-02-03 06:00:00+00:00,CBSE,23527.18,23720.00,23208.01,23314.93,13387.860474,364786,23449.094528


In [50]:
ohlcv_df = btc_data.drop(columns=['exchange','trade_count','vwap'])
ohlcv_df

,open,high,low,close,volume
timestamp,,,,,
2020-01-01 06:00:00+00:00,7174.33,7186.18,7089.00,7108.03,1357.688568
2020-01-02 06:00:00+00:00,6945.02,7281.24,6854.67,7203.04,4385.672853
2020-01-03 06:00:00+00:00,7203.04,7401.20,7176.07,7334.45,9593.744578
2020-01-04 06:00:00+00:00,7333.95,7398.00,7263.32,7348.63,3788.097227
2020-01-05 06:00:00+00:00,7355.40,7580.00,7343.17,7528.70,3362.380611
...,...,...,...,...,...
2023-02-01 06:00:00+00:00,23117.38,24262.18,22760.00,23809.86,21257.753630
2023-02-02 06:00:00+00:00,23809.87,24148.37,23367.87,23527.78,13127.923248
2023-02-03 06:00:00+00:00,23527.18,23720.00,23208.01,23314.93,13387.860474


In [51]:
ohlc_df = ohlcv_df.drop(columns=['volume'])
ohlc_df

,open,high,low,close
timestamp,,,,
2020-01-01 06:00:00+00:00,7174.33,7186.18,7089.00,7108.03
2020-01-02 06:00:00+00:00,6945.02,7281.24,6854.67,7203.04
2020-01-03 06:00:00+00:00,7203.04,7401.20,7176.07,7334.45
2020-01-04 06:00:00+00:00,7333.95,7398.00,7263.32,7348.63
2020-01-05 06:00:00+00:00,7355.40,7580.00,7343.17,7528.70
...,...,...,...,...
2023-02-01 06:00:00+00:00,23117.38,24262.18,22760.00,23809.86
2023-02-02 06:00:00+00:00,23809.87,24148.37,23367.87,23527.78
2023-02-03 06:00:00+00:00,23527.18,23720.00,23208.01,23314.93


In [52]:
ichimoku_df = TA.ICHIMOKU(ohlcv_df, tenkan_period= 20, kijun_period= 60, senkou_period= 120, chikou_period= 30)
ichimoku_df

,TENKAN,KIJUN,senkou_span_a,SENKOU,CHIKOU
timestamp,,,,,
2020-01-01 06:00:00+00:00,NaN,NaN,NaN,NaN,9377.26
2020-01-02 06:00:00+00:00,NaN,NaN,NaN,NaN,9300.14
2020-01-03 06:00:00+00:00,NaN,NaN,NaN,NaN,9323.50
2020-01-04 06:00:00+00:00,NaN,NaN,NaN,NaN,9280.49
2020-01-05 06:00:00+00:00,NaN,NaN,NaN,NaN,9246.02
...,...,...,...,...,...
2023-02-01 06:00:00+00:00,21523.265,20267.79,17429.1500,20337.285,NaN
2023-02-02 06:00:00+00:00,22247.755,20267.79,17455.8475,20337.285,NaN
2023-02-03 06:00:00+00:00,22316.095,20267.79,17455.8475,20337.285,NaN


In [53]:
ichimoku_df.hvplot()

:NdOverlay   [Variable]
   :Curve   [timestamp]   (value)

In [55]:
volume_splice_df['signal'] = np.nan

volume_splice_df.loc[(volume_splice_df['vwap']>= volume_splice_df['close']), 'vwap_signal'] = 1

volume_splice_df.loc[(volume_splice_df['vwap'] < volume_splice_df['close']), 'vwap_signal'] = -1

volume_splice_df["Entry/Exit"] = volume_splice_df["vwap_signal"].diff()

volume_splice_df['vwap_diff_from_close'] = volume_splice_df['close'] - volume_splice_df['vwap']

volume_splice_df.hvplot()

NameError: name 'volume_splice_df' is not defined

In [ ]:
volume_splice_df.hvplot()

In [ ]:
btc_data.hvplot()

In [ ]:
# Keep data from only CBSE exchange
btc_data = btc_data[btc_data['exchange'] == 'CBSE'] 

# keep only the daily close data column
btc_data = btc_data.filter(['close'])

# rename our close column to BTC
btc_data.rename(columns={'close':'BTC'}, inplace=True)

# keep only the date part of our timestamp index
btc_data.index = btc_data.index.map(lambda timestamp : timestamp.date)

# Clean SPY data
#spy_data = spy_data.filter(['close'])
#spy_data.rename(columns={'close':'SPY'}, inplace=True)
#spy_data.index = spy_data.index.map(lambda timestamp : timestamp.date)

In [ ]:
btc_data.hvplot()

In [ ]:
# Starting out my signals dataframe
signals_df = four_hr_ohlcv.loc[:,['close']]
# Generating returns from the BTC close prices using pct_change
signals_df['Actual Returns'] = signals_df['close'].pct_change().dropna()
# Adding more indicators to the sisgnals_df
signals_df['RSI63'] = TA.RSI(four_hr_ohlcv,63)
signals_df.dropna()
# Checking the RSI plot
RSI63 = signals_df['RSI63'].hvplot()



In [ ]:
# making the sma  rolling window for volume
volume_sma = 7
signals_df['volume'] = four_hr_ohlcv['volume']

# making the sma based on the rolling window for volume
signals_df['Volume SMA 7'] = signals_df['volume'].rolling(window=volume_sma).mean()
signals_df.dropna()

# Checking the Volume SMA plot
Volume_SMA_7 = signals_df['Volume SMA 7'].hvplot()
Volume = signals_df['volume'].hvplot.bar()
Volume

